# 5. ネットワーク分析
## 5.4 分析事例: Webサイトの行動履歴

### 5.4.1 準備


In [ ]:
# コード5.1 必要なモジュールの読み込み
import networkx as nx
import gzip
from networkx.algorithms.community.centrality import girvan_newman
from networkx.algorithms.community import greedy_modularity_communities

%precision 4

準備として[Wikipedia Clickstreamデータセット](https://dumps.wikimedia.org/other/clickstream/readme.html)から `clickstream-jawiki-2022-01.tsv.gz`をダウンロードしてください．  
以下のコードは，ダウンロードしたファイルをこのノートブックからアクセスできる場所に置いてから実行してください．
- Google Colaboratoryを使用する場合は2章を参考にしてください．
- パソコン上のjupyterで実行する場合はこのipynbと同じフォルダに置いてください．

In [ ]:
PATH = "./"  # ファイルの置き場所

In [ ]:
# コード5.2 tsv.gzからの無向グラフの作成
g = nx.Graph()
with gzip.open(PATH + "clickstream-jawiki-2022-01.tsv.gz", mode="rt", encoding="utf-8") as f:
    for line in f:
        src, dst, kind, num = line.strip().split("\t")
        num = int(num)
        if kind == "link":
            g.add_edge(src, dst)
print(g)

In [ ]:
# コード5.3
# 圧縮していないtsvファイルを直接読み込む場合は以下のコメントアウトを外して実行してください

# g_tsv = nx.Graph()
# with open(PATH + "clickstream-jawiki-2022-01.tsv", mode="r", encoding="utf-8") as f:
#     for line in f:
#         src, dst, kind, num = line.strip().split("\t")
#         num = int(num)
#         if kind == "link":
#             g_tsv.add_edge(src, dst)
# print(g_tsv)

In [ ]:
# コード5.4 tsv.gzからの有向グラフの作成
g_directed = nx.DiGraph()
with gzip.open(PATH + "clickstream-jawiki-2022-01.tsv.gz", mode="rt", encoding="utf-8") as f:
    for line in f:
        src, dst, kind, num = line.strip().split('\t')
        num = int(num)
        if kind == "link":
            g_directed.add_edge(src, dst)
print(g_directed)

In [ ]:
# コード5.5 グラフにノードが含まれているかの確認
print("データサイエンス" in g)
print("データ科学" in g)

In [ ]:
# コード5.6 「データサイエンス」を中心としたエゴグラフの作成
g_ds = nx.ego_graph(g, "データサイエンス", radius=2, center=True)
print(g_ds)

In [ ]:
# コード5.7 エゴグラフに「データサイエンス」が含まれているかの確認
print("データサイエンス" in g_ds )

### 5.4.2 ノードの分析

In [ ]:
# コード5.8 次数中心性の算出
cent_d = nx.degree_centrality(g_ds)

In [ ]:
# コード5.9 次数中心性の上位10件の表示
display(sorted(cent_d.items(), key=lambda t: -t[1])[:10])

In [ ]:
# コード5.10 近接中心性の算出と上位10件の表示
cent_c = nx.closeness_centrality(g_ds)
display(sorted(cent_c.items(), key=lambda t: -t[1])[:10])

In [ ]:
# コード5.11 媒介中心性の算出と上位10件の表示
cent_b = nx.betweenness_centrality(g_ds)
display(sorted(cent_b.items(), key=lambda t: -t[1])[:10])

In [ ]:
# コード5.12 PageRankの算出と上位10件の表示
pr = nx.pagerank(g_ds, alpha=0.85)
display(sorted(pr.items(), key=lambda t: -t[1])[:10])

In [ ]:
# コード5.13 Personalized PageRankの算出と上位10件の表示
pr_biased = nx.pagerank(g_ds, alpha=0.85, personalization={"統計学": 0.1})
display(sorted(pr_biased.items(), key=lambda t: -t[1])[:10])

In [ ]:
# コード5.14 有向グラフからのエゴグラフの作成
g_ds_directed = nx.ego_graph(g_directed, "データサイエンス", radius=2, center=True, undirected=True)
print(g_ds_directed)

In [ ]:
# コード5.15 有向グラフでのPageRankの算出と上位10件の表示
pr_biased = nx.pagerank(g_ds_directed, alpha=0.85, personalization={"統計学": 0.1})
display(sorted(pr_biased.items(), key=lambda t: -t[1])[:10])

### 5.4.3 コミュニティ抽出

In [ ]:
# コード5.16 Girvan-Newmanアルゴリズムによるコミュニティ抽出
n = 4
comp = girvan_newman(g_ds)
for i, clusters in zip(range(n-1), comp):
    pass
for c in sorted(clusters, key=len, reverse=True):
    print(len(c), sorted(c, key=lambda el: -cent_d[el])[:5])

In [ ]:
# コード5.17 モジュラリティに基づくグラフクラスタリングによるコミュニティ抽出
clusters = greedy_modularity_communities(g_ds)
for c in clusters:
    print(len(c), sorted(c, key=lambda el: -cent_d[el])[:5])